It works fine now. Need testing on multiple items added to test list.
This is a generic version of the AutoPurchaser that lets you create your own version for any website you choose.

In [ ]:
#IMPORTS
from env import  *
import requests as req
import sys
from bs4 import BeautifulSoup as bs
import re
import os
from twilio.rest import Client
from typing import List
from pathlib import Path
import time

In [ ]:
class WebStoreCamper():
    '''
    Allows you to build a program that will watch a sold out product for you in a webstore and automatically alert you when
    it is in stock. Can be set to send you an alert via text message using Twillo or to automatically buy the item for
    you when it becomes available. It will require a bit of knowledge about html and css tags along with the ability
    to use a few basic features of beautifulsoup for scraping info from a webpage.
    This is done by creating a CamperStockChecker to tell the WebStoreCamper how to know when an item is in stock by
    searching for specific css or html tags and looking for specific information. For example, on NewEgg you can do this
    searching for a button with class='btn btn-primary btn-wide'. If thats there, then the item is in stock since
    thats the name of the big yellow button that says IN STOCK on it.
    Lastly you will tell the WebStoreCamper which urls you want it to watch and what actions it should take (alert and/or
    purchase) when the product listed at that url comes back in stock.
    Also you should not include sensitve information in your program. Use environment variables for stuff you want to
    keep secret.
    
    Steps to use (with example code!):
    1. CREATE A WebStoreCamper OBJECT
        camper = WebStoreCamper('MyBestBuyCamper')
    
    2. INITIATE ALERTS AND/OR PURCHASES:
        my_sid= '83894sgghsdfhsdghgs9f9fqgg24y'
        my_token = '383qqrgqt234f6wf7wghncnq3y'
        t_num = '+15556667777'
        my_cell = '+12223334444'
        msg = 'Dude this product finally came in!'
        camper.initiate_text_alerts(my_sid, my_token, t_num, my_cell, msg)
        
        usr = 'MemeBasedUserName'
        pwd = 'Secretpassnooneknows'
        camper.initialize_purchases(usr, pwd)
        
    3. CREATE CHECK NVENTORY METHOD AND SET IT:
        def my_check_method(soup):
            a= soup.find(some_tag='something')
            if a:
                return True
            else:
                return False
            
        camper.set_check_inventory_method(my_check_method)
    
        
    5. ADD URLS OF PRODUCTS YOU WISH TO WATCH AND SET IF YOU WANT ALERTS AND PURCHASES
        client.add_to_watchlist(url = 'http://bestbuy.com/some-cool-tv/', alert = True, purchases = 1)
        client.add_to_watchlist(url = 'http://bestbuy.com/some-cool-blender/', alert = True, purchases = 2)
        client.add_to_watchlist(url = 'http://bestbuy.com/some-cool-computer/', alert = True
        #note this last one will only alert if the computer becomes available it won't buy since purchases not set
        
    6. START IT UP
        client.begin_camping()
        
            
        
    param: name - The name you wish to refer to this camper by. ex. "NewEggCamper"
    '''
    
    def __init__(self, name:str):
        '''
         Creates a camper object for you to configure. 
         :param name:  The name you wish to refer to this camper by. ex. "BlakesBestBuyCamper"
         :param useragent: No need to change this unless there is a problem. It is to trick the website into thinking
                             it is being accessed via a web browser and not a webscraper when checking stock.
        '''
        self.name = name
        self.user_agent = str('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) ' 
                              + 'Chrome/87.0.4280.66 Safari/537.36')
        self.watched_products_list = []
        self.check_inventory = None
        
        
    def initiate_text_alerts(self,
                             twilio_sid:str,
                             twilio_auth_token:str,
                             twilio_number:str,
                             message:str):
        '''
        Sets the ability to recieve text alerts when an item is in stock.
        :param twilio_sid: The sid for your twillio project
        :param twilio_auth_token: The auth token for your twillio project
        :param twilio_number: The phone number belonging to the twilio project you are using
                                entered as a string with format: [+][country code][phone number including area code]
                                An example number in the USA would be '+15554443333'
        :param receiving_number: The cell number you wish to recieve an alert text message at entered as a 
                                    string with format: [+][country code][phone number including area code]
                                    An example number in the USA would be '+15554443333'
        :param message: The message you would like to send when the item becomes available; entered as a string. 
                        For example: "The itme I've been waiting for is finally here!" Note that the link to the product
                        will be sent along with the text message so that you can quickly purchase it 
        '''
        if not isinstance(twilio_number, str) or len(twilio_number) not in range(11,18):
            raise Exception('The phone number belonging to the twilio project you are using must be entered as a '
                            + 'string with format: [+][country code][phone number including area code]')
        if not isinstance(receiving_number, str) or len(receiving_number) not in range(11,18):
            raise Exception('The phone number belonging to the message reciever must be entered as a '
                            + 'string with format: [+][country code][phone number including area code]')
        
        self.twilio_number = twilio_number,
        self.receiving_number = receiving_number
        self.twilio_sid = twilio_sid
        self.twilio_auth_token = twilio_auth_token
        self.message = message
        
        
    def initialize_purchases(self,
                        username: str,
                        password: str):
        '''
        :param purchase: True/False, if True must include the username and password parameters
        :param username: the username for the website where the item will be purchased from.
        :param password: the password for the website where the item will be purchased from.
        '''
        if not isinstance(username, str):
            raise Exception('username must be an integer value')
        if not isinstance(password, str):
            raise Exception('password must be an integer value')
        if not isinstance(num_purchases, int):
            raise Exception('num_purchases must be an integer value')

        self.username = username
        self.password = password
        
        
        
    def begin_camping(self):
        '''
        Searches on new egg for the search_term then monitors the stock of items on the first
        results page that have descriptions matching the search term. Alerts will be sent for each
        item as it becomes available. Items will be purchased as they become available until the number
        of items purchases is equal to self.num_purchases
        '''
        while True:
            if len(self.watched_products_list) < 1:
                #raise Exception('You supplied an empty list to watch_multiple_items. Please enter a list of url strings')
                print('No items to watch at the moment')
                time.sleep(2)
                continue
            while self.watched_products_list:
                for product in self.watched_products_list:
                    self.url = product['url']
                    html = self._get_html()
                    soup = self._get_soup(html)
                    if self.check_inventory(soup):
                        #Item in stock so now process purchase and/or alerts
                        print(f'Item in stock!\n Link: {self.url}\n\n')
                        if product['alert']:
                            print(f"Item in stock. Sending alert to {self.receiving_number}")
                            self.receiving_number = product['receiving_number']
                            try:
                                message = f'{self.message} Link: {self.url}'
                                self._send_alert(message)
                            except Exception as err:
                                print(f'The alert failed to send: {err}')
                                print(sys.exc_info()[0])
                            else:
                                print('Alert sent')
                        #remove url from list now that purchase/alerts processed 
                        print(self.watched_products_list)
                        self.watched_products_list = [i for i in self.watched_products_list if i['url'] != self.url]
                    else:
                        print(f'Item  not avaialble. Link: {self.url}')
            
        

            
    def set_check_inventory_method(self, inv_method):
        '''
        Define a function that accepts a soup object as its only parameter and uses that to decide if a
        product is in stock or not. The return signature be either True or False
        '''
        self.check_inventory = inv_method

        
        
    def add_to_watchlist(self,
                         url:str=None,
                         alert=False,
                         receiving_number:str=None):
        '''
        :param url: The url of the product you want to camp
        :param alert: Set to True if you wish to recieve a text when this item is in stock. You must have have
                        initiated and configured alerts for this to work.
        :param purchases: Setting this to any positive value will attempt to purchase the product that many times. You
                            must have both initiated purchases and configured purchases for this to work.
        '''
        #if not isinstance(receiving_number, str) or len(receiving_number) not in range(11,18):
         #   raise Exception('The phone number belonging to the message reciever must be entered as a '
         #                   + 'string with format: [+][country code][phone number including area code]')
        #clear this next line its only for testing
        self.watched_products_list = []
        self.watched_products_list.append({'url':url, 'alert':alert, 'receiving_number':receiving_number})
        
        
        
    def _send_alert(self, msg):
        twilio_client = Client(self.twilio_sid, self.twilio_auth_token)
        send_message = twilio_client.messages.create(from_= self.twilio_number, body = self.message, to = self.receiving_number)
        print(f'{send_message} \n sent to {self.receiving_number}\n')

        
        
    def _get_soup(self, html):
        soup = bs(html, 'html.parser')
        return soup
        
    def _get_html(self):
        headers = {'User-Agent': self.user_agent}
        page = req.get(self.url, headers=headers)
        if page.status_code != 200:
            raise Exception(f"The page returned a status code that was not 200. Status code was {page.status_code}")
        return page.content        
        
    def set_user_agent(self, new_agent:str):
        '''
        No need to use this method unless there is some kind of problem relating to the useragent. 
        It is to trick the website into thinking it is being accessed via a web browser and not a
        webscraper when the program checks stock on the website.
        :param new_agent: The new agent you wish to be used when sending http requests 
        '''
        self.user_agent = new_agent    
        
        
        
    def test(self):
        pass

In [ ]:
#you can use this to test if you create an env.py file and define TWILIO_SID, TWILIO_TOKEN, TWILIO_NUMBER, and MY_CELL
#as variables in it since I've got env imported at the very top of this notebook. 

camper = WebStoreCamper("BestBuyCamper")
sid = TWILIO_SID
auth_token = TWILIO_TOKEN
twillio_number = TWILIO_NUMBER
receiving_number = MY_CELL
msg = 'This alert is to let you know the product you wanted to watch is now available for purchase.'
camper.initiate_text_alerts(sid, auth_token, twillio_number, msg)

def best_buy_checker(soup):
    a = soup.find('button', class_="btn btn-primary btn-lg btn-block btn-leading-ficon add-to-cart-button")
    if a is not None:
        return True
    else:
        return False
    
camper.set_check_inventory_method(best_buy_checker)
camper.add_to_watchlist('https://www.bestbuy.com/site/samsung-4-5-cu-ft-5-cycle-front-load-washer-with-vibration-reduction-technology-white/6416171.p?skuId=6416171',
                        True,
                        receiving_number)
print(camper.watched_products_list)
camper.begin_camping()